In [ ]:
import openai

client = openai.OpenAI()
messages = []


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    message = response.choices[0].message.content
    messages.append({
        "role": "assistant", 
        "content": message
    })
    print(f"AI: {message}")


while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({
            "role": "user", 
            "content": message
        })
        print(f"User: {message}")
        call_ai()
        
        


NameError: name 'f' is not defined

In [8]:
for choice in response.choices:
    print(choice)

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="get_weather('Greece')", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))


In [9]:
message = response.choices[0].message.content

message

"get_weather('Greece')"